# Module 1 (Part 2): Automatic Differentiation


**Exercise:** [![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=flat-square&logo=googlecolab)](https://colab.research.google.com/github/kks32-courses/ut-portugal/blob/main/docs/00-mlp/ad.ipynb)
**Solution:** [![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=flat-square&logo=googlecolab)](https://colab.research.google.com/github/kks32-courses/ut-portugal/blob/main/docs/00-mlp/ad.ipynb)

**Slides:** [![View PDF](https://img.shields.io/badge/View-PDF-red?style=flat-square&logo=googledocs&logoColor=white)](https://github.com/kks32-courses/ut-portugal-sciml/raw/main/docs/00-mlp/ad-slides.pdf)

## Introduction: Why Do We Need Better Ways to Compute Derivatives?

In scientific computing, we constantly need derivatives. Whether we're solving optimization problems, training neural networks, or inverting geophysical data, derivatives are the mathematical engine that drives our algorithms forward.

Traditionally, we've had three approaches to compute derivatives, each with fundamental limitations:

![Modes of differentiation](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/differentiation.png)

## The Four Modes of Differentiation

### 1. Manual Differentiation
The traditional calculus approach: derive formulas by hand using calculus rules.

**Pros**: 
- Exact derivatives
- Allows optimization during derivation

**Cons**: 
- Error-prone and time-consuming
- Not scalable for complex functions
- Impractical for functions with thousands of parameters

### 2. Symbolic Differentiation
Let computers apply calculus rules symbolically, like Mathematica or SymPy.

**Pros**:
- Provides exact, closed-form expressions

**Cons**:
- **Expression swell**: derivatives can become exponentially large
- Not feasible when symbolic form isn't available
- Computationally expensive for complex expressions

### 3. Numerical Differentiation
Approximate derivatives using finite differences:
$$ f'(x) \approx \frac{f(x + h) - f(x - h)}{2h} $$

**Pros**:
- Simple to implement
- Works for any function you can evaluate

**Cons**:
- **Accuracy issues**: prone to rounding errors
- **Step size dilemma**: too large → truncation error, too small → round-off error
- **Computational cost**: requires multiple function evaluations

### 4. Automatic Differentiation (AD)
The revolutionary approach: compute exact derivatives alongside function evaluation.

**Pros**:
- **Exact derivatives** (up to machine precision)
- **Efficient**: computational cost proportional to function evaluation
- **General**: works for any differentiable function expressible as code

**Cons**:
- Requires specialized libraries
- Can be memory-intensive for some applications

## The Core Insight: Functions Are Computational Graphs

Every computer program that evaluates a mathematical function can be viewed as a **computational graph**. Consider this simple function:

In [ ]:
def f(x1, x2):
    y = x1**2 + x2
    return y

This creates a computational graph where each operation is a node:

![AD graph](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad1.png)

We can decompose this into elementary operations and assign intermediate variables:

![AD graph variables](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad2.png)

This decomposition is the key insight that makes automatic differentiation possible.

![AD forward pass](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad3.png)

## Forward Mode Automatic Differentiation

Forward mode AD computes derivatives by propagating derivative information **forward** through the computational graph, following the same path as the function evaluation.

![AD forward evaluation](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/forward-mode-ad.png)

### Forward Mode: Computing $\frac{\partial y}{\partial x_1}$

Starting with our function $y = x_1^2 + x_2$, let's trace through the computation:

1. **Seed the input**: Set $\dot{x}_1 = 1$ and $\dot{x}_2 = 0$ (we're differentiating w.r.t. $x_1$)

2. **Forward propagation**:
   - $v_1 = x_1^2$, so $\dot{v}_1 = 2x_1 \cdot \dot{x}_1 = 2x_1 \cdot 1 = 2x_1$
   - $y = v_1 + x_2$, so $\dot{y} = \dot{v}_1 + \dot{x}_2 = 2x_1 + 0 = 2x_1$

3. **Result**: $\frac{\partial y}{\partial x_1} = 2x_1$

### Forward Mode: Computing $\frac{\partial y}{\partial x_2}$

To get the derivative w.r.t. $x_2$, we seed differently:

1. **Seed the input**: Set $\dot{x}_1 = 0$ and $\dot{x}_2 = 1$

2. **Forward propagation**:
   - $v_1 = x_1^2$, so $\dot{v}_1 = 2x_1 \cdot \dot{x}_1 = 2x_1 \cdot 0 = 0$
   - $y = v_1 + x_2$, so $\dot{y} = \dot{v}_1 + \dot{x}_2 = 0 + 1 = 1$

3. **Result**: $\frac{\partial y}{\partial x_2} = 1$

**Key insight**: Forward mode requires one pass per input variable to compute all partial derivatives.

## Reverse Mode Automatic Differentiation

Reverse mode AD (also called **backpropagation**) computes derivatives by propagating derivative information **backward** through the computational graph.

![Reverse mode AD](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad4.png)

### The Backward Pass Algorithm

1. **Forward pass**: Compute function values and store intermediate results
2. **Seed the output**: Set $\bar{y} = 1$ (derivative of output w.r.t. itself)
3. **Backward pass**: Use the chain rule to propagate derivatives backward

![Chain rule AD](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad5.png)

Let's trace through our example:

![Chain rule AD steps](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad6.png)

![Final chain rule AD](https://raw.githubusercontent.com/kks32-courses/ut-portugal-sciml/refs/heads/main/docs/00-mlp/figs/ad7.png)

### Computing All Partial Derivatives in One Pass

The beauty of reverse mode is that it computes **all** partial derivatives in a single backward pass:

1. **Forward pass**: $y = x_1^2 + x_2$ (store intermediate values)

2. **Backward pass with $\bar{y} = 1$**:
   - $\frac{\partial y}{\partial x_1} = \frac{\partial y}{\partial v_1} \cdot \frac{\partial v_1}{\partial x_1} = 1 \cdot 2x_1 = 2x_1$
   - $\frac{\partial y}{\partial x_2} = \frac{\partial y}{\partial x_2} = 1$

**Key insight**: Reverse mode computes gradients w.r.t. all inputs in a single backward pass!

## When to Use Forward vs Reverse Mode

The choice depends on the structure of your problem:

- **Forward Mode**: Efficient when **few inputs, many outputs** (e.g., $f: \mathbb{R}^n \to \mathbb{R}^m$ with $n \ll m$)
- **Reverse Mode**: Efficient when **many inputs, few outputs** (e.g., $f: \mathbb{R}^n \to \mathbb{R}^m$ with $n \gg m$)

In machine learning, we typically have millions of parameters (inputs) and a single loss function (output), making reverse mode the natural choice.

## The Mathematical Foundation

Automatic differentiation works because of a fundamental theorem:

**Chain Rule**: For composite functions $f(g(x))$:
$$\frac{d}{dx}f(g(x)) = f'(g(x)) \cdot g'(x)$$

By systematically applying the chain rule to each operation in a computational graph, AD can compute exact derivatives for arbitrarily complex functions.

## Automatic Differentiation in Practice: PyTorch

Let's see how automatic differentiation works in PyTorch:

In [1]:
import torch

# Define variables that require gradients
x1 = torch.tensor(2.0, requires_grad=True)
x2 = torch.tensor(3.0, requires_grad=True)

# Define the function
y = x1**2 + x2

# Compute gradients using reverse mode AD
y.backward()

# Access the computed gradients
print(f"dy/dx1: {x1.grad.item()}")  # Should be 2*x1 = 4.0
print(f"dy/dx2: {x2.grad.item()}")  # Should be 1.0

dy/dx1: 4.0
dy/dx2: 1.0


### A More Complex Example: Neural Network

In [2]:
import torch
import torch.nn as nn

# Simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 3)
        self.layer2 = nn.Linear(3, 1)
    
    def forward(self, x):
        x = torch.tanh(self.layer1(x))
        x = self.layer2(x)
        return x

# Create network and data
net = SimpleNet()
x = torch.tensor([[1.0, 2.0]], requires_grad=True)
target = torch.tensor([[0.5]])

# Forward pass
output = net(x)
loss = ((output - target)**2).mean()

# Backward pass - computes gradients for ALL parameters
loss.backward()

# Access gradients
for name, param in net.named_parameters():
    print(f"{name}: gradient shape {param.grad.shape}")

layer1.weight: gradient shape torch.Size([3, 2])
layer1.bias: gradient shape torch.Size([3])
layer2.weight: gradient shape torch.Size([1, 3])
layer2.bias: gradient shape torch.Size([1])


The power of automatic differentiation becomes clear: PyTorch automatically computes gradients for all parameters in the network, regardless of how complex the architecture becomes.

## Computational Considerations

### Memory vs Computation Trade-offs

**Forward Mode**:
- Memory: O(1) additional storage
- Computation: O(n) for n input variables

**Reverse Mode**:
- Memory: O(computation graph size)
- Computation: O(1) for any number of input variables

### Modern Optimizations

1. **Checkpointing**: Trade computation for memory by recomputing intermediate values
2. **JIT compilation**: Compile computational graphs for faster execution
3. **Parallelization**: Distribute gradient computation across multiple devices

## Summary: The AD Revolution

Automatic differentiation has revolutionized scientific computing by making gradients:

1. **Ubiquitous**: Available for any differentiable computation
2. **Exact**: No approximation errors (up to machine precision)
3. **Efficient**: Computational cost proportional to function evaluation
4. **Automatic**: No manual derivation required

This has enabled new approaches to:
- Machine learning (deep learning wouldn't exist without AD)
- Scientific computing (differentiable simulators)
- Optimization (gradient-based methods for complex problems)
- Parameter estimation (automatic gradient computation for inverse problems)

The combination of automatic differentiation with modern hardware (GPUs, TPUs) has created unprecedented opportunities for scientific discovery through computation.

## Looking Forward

As we move into the era of scientific machine learning, automatic differentiation serves as the mathematical engine that powers:
- Physics-informed neural networks
- Neural differential equations  
- Differentiable simulators
- End-to-end optimization of scientific workflows

The ability to compute gradients automatically through arbitrary computational processes is not just a technical convenience—it's a fundamental capability that's reshaping how we approach scientific problems in the 21st century.